In [60]:
import pandas as pd

import spacy
from spacy import displacy

In [ ]:
# my_spacy = spacy.load("en")
my_spacy = spacy.load("en_core_web_md") # load up spaCy

In [61]:
# test if my_spacy works first
random_sentence = "The pasta was great, but the waiter was rude!"
random_sen_spacy = my_spacy(random_sentence)

In [70]:
# Positive and Negative Adjectives (courtesy of Liu B.)

# Negative Adjectives
neg_lexicons = open('opinion_lexicons/negative_words.txt')
neg_words = []

for word in neg_lexicons:
    neg_words.append(word.strip())

# Positive Adjectives
pos_lexicons = open('opinion_lexicons/positve_words.txt')
pos_words = []

for word in pos_lexicons:
    pos_words.append(word.strip())

In [71]:
len(pos_words) + len(neg_words)

6790

In [76]:
## read in pickled pandas data
import pickle

reviews_pd = pickle.load(open("pickled_data/pickled_reviews_pd.pkl", "rb"))
reviews_pd.head(3)

,Aspect Count,Aspect Polarity,Aspect Term,Category,Category Count,Category Polarity,Review,Review ID
0,1,[negative],[staff],[service],1,[negative],But the staff was so horrible to us.,3121
1,1,[positive],[food],"[food, anecdotes/miscellaneous]",2,"[positive, negative]","To be completely fair, the only redeeming fact...",2777
2,3,"[positive, positive, neutral]","[food, kitchen, menu]",[food],1,[positive],"The food is uniformly exceptional, with a very...",1634


In [94]:
subset = reviews_pd.iloc[0:15,]
subset

,Aspect Count,Aspect Polarity,Aspect Term,Category,Category Count,Category Polarity,Review,Review ID
0,1,[negative],[staff],[service],1,[negative],But the staff was so horrible to us.,3121
1,1,[positive],[food],"[food, anecdotes/miscellaneous]",2,"[positive, negative]","To be completely fair, the only redeeming fact...",2777
2,3,"[positive, positive, neutral]","[food, kitchen, menu]",[food],1,[positive],"The food is uniformly exceptional, with a very...",1634
3,3,"[positive, positive, neutral]","[food, kitchen, menu]",[service],1,[positive],Where Gabriela personaly greets you and recomm...,2534
4,3,"[positive, positive, neutral]","[food, kitchen, menu]",[anecdotes/miscellaneous],1,[positive],"For those that go once and don't enjoy it, all...",583
5,2,"[positive, positive]","[food, perks]","[food, service]",2,"[positive, positive]","Not only was the food outstanding, but the lit...",2846
6,2,"[positive, positive]","[food, perks]","[food, price]",2,"[negative, negative]",It is very overpriced and not very tasty.,1571
7,4,"[positive, positive, neutral, neutral]","[orrechiete with sausage and chicken, waiters,...","[food, service]",2,"[positive, positive]",Our agreed favorite is the orrechiete with sau...,1458
8,1,[positive],[Bagels],[food],1,[positive],The Bagels have an outstanding taste with a te...,3161
9,1,[positive],[food],[food],1,[positive],Nevertheless the food itself is pretty good.,2391


In [ ]:
# 

In [98]:
absa_classifier_unsupervised = []

for row in subset.itertuples():

    data_dict = {}
    review = row[7]
    #print(review)
    data_dict["Review"] = review
    
    # convert review to spaCy object
    review_spaCy = my_spacy(review)
    print()
    for tok in review_spaCy:
        print(tok.text, tok.pos_, tok.dep_)
    
    asp_terms = []
    asp_pols = []
    
    for token in review_spaCy:
        if (token.pos_ == "NOUN"):
            asp_terms.append(token.text)
        if (token.dep_ == "acomp" or token.dep_ == "amod"):
            if (token.text in pos_words):
                asp_pols.append("positive")
            elif (token.text in neg_words):
                asp_pols.append("negative")
            else:
                asp_pols.append("neutral")
    data_dict["Aspect Term"] = asp_terms
    data_dict["Aspect Polarity"] = asp_pols
    
    absa_classifier_unsupervised.append(data_dict)


But CCONJ cc
the DET det
staff NOUN nsubj
was VERB ROOT
so ADV advmod
horrible ADJ acomp
to ADP prep
us PRON pobj
. PUNCT punct

To PART aux
be VERB advcl
completely ADV advmod
fair ADJ acomp
, PUNCT punct
the DET det
only ADJ advmod
redeeming VERB amod
factor NOUN nsubj
was VERB ROOT
the DET det
food NOUN attr
, PUNCT punct
which ADJ nsubj
was VERB relcl
above ADP prep
average ADJ amod
, PUNCT punct
but CCONJ cc
could VERB aux
n't ADV neg
make VERB conj
up PART prt
for ADP prep
all ADJ predet
the DET det
other ADJ amod
deficiencies NOUN pobj
of ADP prep
Teodora PROPN pobj
. PUNCT punct

The DET det
food NOUN nsubj
is VERB ROOT
uniformly ADV advmod
exceptional ADJ acomp
, PUNCT punct
with ADP prep
a DET det
very ADV advmod
capable ADJ amod
kitchen NOUN pobj
which ADJ nsubj
will VERB aux
proudly ADV advmod
whip VERB relcl
up PART prt
whatever ADJ dobj
you PRON nsubj
feel VERB ccomp
like ADP prep
eating VERB pcomp
, PUNCT punct
whether ADP mark
it PRON nsubj
's VERB advcl
on ADP prep
th

[{'Aspect Polarity': ['negative'],
  'Aspect Term': ['staff'],
  'Review': 'But the staff was so horrible to us.'},
 {'Aspect Polarity': ['positive'],
  'Aspect Term': ['factor', 'food', 'deficiencies'],
  'Review': "To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora."},
 {'Aspect Polarity': ['positive'],
  'Aspect Term': ['food', 'kitchen', 'menu'],
  'Review': "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not."}]

In [96]:
classifier_pd = pd.DataFrame(absa_classifier_unsupervised)
classifier_pd.head(15)

,Aspect Polarity,Aspect Term,Review
0,[negative],[staff],But the staff was so horrible to us.
1,[positive],"[factor, food, deficiencies]","To be completely fair, the only redeeming fact..."
2,[positive],"[food, kitchen, menu]","The food is uniformly exceptional, with a very..."
3,[],[what],Where Gabriela personaly greets you and recomm...
4,[],[],"For those that go once and don't enjoy it, all..."
5,[positive],"[food, perks]","Not only was the food outstanding, but the lit..."
6,[negative],[],It is very overpriced and not very tasty.
7,[positive],"[favorite, orrechiete, sausage, chicken, waite...",Our agreed favorite is the orrechiete with sau...
8,[],"[Bagels, taste, texture]",The Bagels have an outstanding taste with a te...
9,[positive],[food],Nevertheless the food itself is pretty good.


In [92]:
reviews_pd.head(10)

,Aspect Count,Aspect Polarity,Aspect Term,Category,Category Count,Category Polarity,Review,Review ID
0,1,[negative],[staff],[service],1,[negative],But the staff was so horrible to us.,3121
1,1,[positive],[food],"[food, anecdotes/miscellaneous]",2,"[positive, negative]","To be completely fair, the only redeeming fact...",2777
2,3,"[positive, positive, neutral]","[food, kitchen, menu]",[food],1,[positive],"The food is uniformly exceptional, with a very...",1634
3,3,"[positive, positive, neutral]","[food, kitchen, menu]",[service],1,[positive],Where Gabriela personaly greets you and recomm...,2534
4,3,"[positive, positive, neutral]","[food, kitchen, menu]",[anecdotes/miscellaneous],1,[positive],"For those that go once and don't enjoy it, all...",583
5,2,"[positive, positive]","[food, perks]","[food, service]",2,"[positive, positive]","Not only was the food outstanding, but the lit...",2846
6,2,"[positive, positive]","[food, perks]","[food, price]",2,"[negative, negative]",It is very overpriced and not very tasty.,1571
7,4,"[positive, positive, neutral, neutral]","[orrechiete with sausage and chicken, waiters,...","[food, service]",2,"[positive, positive]",Our agreed favorite is the orrechiete with sau...,1458
8,1,[positive],[Bagels],[food],1,[positive],The Bagels have an outstanding taste with a te...,3161
9,1,[positive],[food],[food],1,[positive],Nevertheless the food itself is pretty good.,2391
